In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
%matplotlib inline

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
crime = pd.read_csv('boston_crime.csv')

In [13]:
crime.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I192028175,3831,Motor Vehicle Accident Response,M/V - LEAVING SCENE - PROPERTY DAMAGE,E5,717,NaN,2019-04-14 16:00:00,2019,4,Sunday,16,Part Three,CENTRE LN,42.270945,-71.160865,"(42.27094455, -71.16086461)"
1,I192028168,3410,Towed,TOWED MOTOR VEHICLE,C11,340,NaN,2019-04-16 20:25:00,2019,4,Tuesday,20,Part Three,DRAPER ST,42.305407,-71.065471,"(42.30540715, -71.06547135)"
2,I192028167,3006,Medical Assistance,SICK/INJURED/MEDICAL - PERSON,B3,459,NaN,2019-04-16 20:57:00,2019,4,Tuesday,20,Part Three,GLENARM ST,42.304385,-71.079945,"(42.30438536, -71.07994459)"
3,I192028166,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,B2,291,NaN,2019-04-16 20:52:00,2019,4,Tuesday,20,Part One,CENTRE ST,42.329554,-71.093574,"(42.32955354, -71.09357444)"
4,I192028166,802,Simple Assault,ASSAULT SIMPLE - BATTERY,B2,291,NaN,2019-04-16 20:52:00,2019,4,Tuesday,20,Part Two,CENTRE ST,42.329554,-71.093574,"(42.32955354, -71.09357444)"


In [6]:
crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378524 entries, 0 to 378523
Data columns (total 17 columns):
INCIDENT_NUMBER        378524 non-null object
OFFENSE_CODE           378524 non-null int64
OFFENSE_CODE_GROUP     378524 non-null object
OFFENSE_DESCRIPTION    378524 non-null object
DISTRICT               376528 non-null object
REPORTING_AREA         378524 non-null object
SHOOTING               1521 non-null object
OCCURRED_ON_DATE       378524 non-null object
YEAR                   378524 non-null int64
MONTH                  378524 non-null int64
DAY_OF_WEEK            378524 non-null object
HOUR                   378524 non-null int64
UCR_PART               378425 non-null object
STREET                 366888 non-null object
Lat                    354784 non-null float64
Long                   354784 non-null float64
Location               378524 non-null object
dtypes: float64(2), int64(4), object(11)
memory usage: 49.1+ MB


In [ ]:
crime_short = crime[['OFFENSE_CODE', 'OFFENSE_CODE_GROUP',
       'OFFENSE_DESCRIPTION', 'REPORTING_AREA', 'SHOOTING',
       'OCCURRED_ON_DATE', 'YEAR', 'MONTH', 'DAY_OF_WEEK', 'HOUR']]

In [ ]:
crime_short['SHOOTING'].replace(np.nan, 0, inplace = True)

In [ ]:
crime_short.to_csv('crime_short.csv', index = False)

In [ ]:
crime_short.head()

In [ ]:
crime_short.info()

In [ ]:
crime_short['OCCURRED_ON_DATE'].nunique()

In [ ]:
def get_column_nunique(df):
    for column in df.columns:
        print(f'{column} has {df[column].nunique()} unique values')
    return

In [ ]:
get_column_nunique(crime_short)

In [15]:
crime_find_district = crime[crime['DISTRICT'].isnull()]

In [7]:
from geopy.geocoders import Nominatim

In [17]:
def find_neighborhood(lat_long, geolocator):
    try:
        
        location = geolocator.reverse(lat_long.strip('(').strip(')'))
        return location.raw['address']['neighbourhood']
    except:
        return 'check'

In [18]:
geolocator = Nominatim()
crime_find_district['Neighborhood'] = crime_find_district['Location'].apply(lambda x: find_neighborhood(x, geolocator))

In [19]:
crime_district = crime[~crime['DISTRICT'].isnull()]
crime_district = crime_district[['OFFENSE_CODE','OFFENSE_CODE_GROUP','DISTRICT','OCCURRED_ON_DATE','DAY_OF_WEEK','HOUR']]

In [20]:
crime_district['OCCURRED_ON_DATE'] = pd.to_datetime(crime_district['OCCURRED_ON_DATE'])
crime_district['DATE'] = crime_district['OCCURRED_ON_DATE'].apply(lambda date:date.date())
crime_district['DATE_HOUR'] = crime_district['OCCURRED_ON_DATE'].apply(lambda date:date.ceil('H'))

In [21]:
crime_district.to_csv('crime_district.csv', index = False)